<a href="https://colab.research.google.com/github/migostro/laboratorio-de-visao/blob/main/2_3_BoundingBoxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
# utilities
from operator import truediv
import os
import time
import numpy as np
from google.colab import files, drive

# image processing
from scipy import ndimage
from skimage import io, exposure, filters, transform
from skimage import data
from skimage import color
from skimage import morphology
from skimage import segmentation, measure
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from sklearn.metrics import jaccard_score

# visualization
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# Helper Functions

In [ ]:
def float_to_int(img):
    return (img*255).astype(np.uint8)

In [ ]:
def display_single(image):
    fig, ax = plt.subplots()
    ax = plt.imshow(image, cmap=plt.cm.gray)
    plt.show()
    plt.close()

In [ ]:
# list_filepaths(): list filepath for every filepath on a root folder
# pre-condition: (root path, empty list)
# post-condition: list with every filepath on root path
def list_filepaths(path, filepaths = []):
    for filename in os.listdir(path):
        if '.csv' not in filename and '.JPG' not in filename: # rejects csvs
            filepath = os.path.join(path, filename)
            if os.path.isfile(filepath): filepaths.append(filepath) # Adiciona apenas caminhos que são de arquivos (que no nosso caso são imagens)
            else: list_filepaths(filepath, filepaths)

    return filepaths

In [ ]:
def crop_image(img, bbox): 
    return img[bbox[0]:bbox[2], bbox[1]:bbox[3]]

#Pipeline

In [ ]:
# 0 FILE READING
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new'
truth_path = root + "/groundTruth_resized"
segmented_path = root + "/segmented_resized"
bbox_path = root + "/bbox_resized"
bbox_truth_path = root + "/bbox_truth_resized"

segmentedData_filepaths = list_filepaths(segmented_path, [])
truthData_filepaths = list_filepaths(truth_path, [])
print(f'# of images: {len(segmentedData_filepaths)}')
print(f'# of images: {len(truthData_filepaths)}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
# of images: 1080
# of images: 160


In [ ]:
scores =	{
  'borrachas': [],
  'cartas': [],
  'celulares': [],
  'conchas': [],
  'copos': [],
  'dados': [],
  'estatuas': [],
  'lapis': [],
  'tubos': [],
  'vasos': [],
}
scores_background =	{
  'branco': [],
  'preto': [],
  'verde': [],
}
scores_env =	{
  'exterior': [],
  'interior': [],
}
scores_time =	{
  'dia': [],
  'noite': [],
}

for idx, img_path in enumerate(segmentedData_filepaths):
    # [0] Image Input
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(segmented_path, '').replace(filename, '')

    # read
    hasTruth = False
    if os.path.isfile(img_path.replace(segmented_path, truth_path).replace('.JPG', '.png')):
        truth = io.imread(img_path.replace(segmented_path, truth_path).replace('.JPG', '.png'))[:,:,0]
        truth = truth > filters.threshold_mean(truth)
        hasTruth = True

        label_truth = measure.label(truth)
        regions_truth = measure.regionprops(label_truth)
        truth_bb = np.zeros(truth.shape)

        if(len(regions_truth) > 1):
            # isolation
            center = (int(truth.shape[0]/2), int(truth.shape[1]/2)) # image center
            most_centered_region = min(regions_truth, key=lambda region: (np.linalg.norm(np.subtract(region.centroid, center))/region.area) ) # minimizes center distance and maximizes object area
            truth_bb[most_centered_region.slice] = 1 # fills bounding box

            # write bounding box
            if not os.path.isdir(bbox_truth_path + middle_path): os.makedirs(bbox_truth_path + middle_path)
            io.imsave(img_path.replace(segmented_path, bbox_truth_path).replace('.JPG', '.png'), float_to_int(most_centered_region.image), check_contrast=False)
        else:
            if(len(regions_truth) > 0): truth_bb[regions_truth[0].slice] = 1 # fills bounding box

            # write bounding box
            if not os.path.isdir(bbox_truth_path + middle_path): os.makedirs(bbox_truth_path + middle_path)
            if(len(regions_truth) > 0): io.imsave(img_path.replace(segmented_path, bbox_truth_path).replace('.JPG', '.png'), float_to_int(regions_truth[0].image), check_contrast=False)
            else: io.imsave(img_path.replace(segmented_path, bbox_truth_path).replace('.JPG', '.png'), np.zeros((1,1)), check_contrast=False)
    # [1] Jaccard Scores
    if hasTruth:
        print(f'[{idx}]', img_path)
        img = io.imread(img_path)

        label = measure.label(img)
        regions = measure.regionprops(label)

        img_bb = np.zeros(img.shape)
        if(len(regions) > 1):
            # isolation
            center = (int(truth.shape[0]/2), int(truth.shape[1]/2)) # image center
            most_centered_region = min(regions, key=lambda region: (np.linalg.norm(np.subtract(region.centroid, center))/region.area) ) # minimizes center distance and maximizes object area
            img_bb[most_centered_region.slice] = 1 # fills bounding box
        elif(len(regions) > 0):
            img_bb[regions[0].slice] = 1 # fills bounding box

        jaccard_idx = jaccard_score(img_bb*1, truth_bb*1, average='micro') # micro p/ 2d image comparison https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html

        curr_class = img_path.split('/')[-3]
        background = img_path.split('/')[-4]
        environment = img_path.split('/')[-5]
        time = img_path.split('/')[-6]
        if curr_class  in scores:            scores[curr_class].append(jaccard_idx)
        if background  in scores_background: scores_background[background].append(jaccard_idx)
        if environment in scores_env:        scores_env[environment].append(jaccard_idx)
        if time        in scores_time:       scores_time[time].append(jaccard_idx)

In [ ]:
# get average jaccard scores
print('[1.1] Average Jaccard Scores by Class')
for key, value in scores.items():
    if value: print(key, np.average(value), np.median(value), len(value))
print('\n[1.2] Average Jaccard Scores by Background')
for key, value in scores_background.items():
    if value: print(key, np.average(value), np.median(value), len(value))
print('\n[1.3] Average Jaccard Scores by Environment')
for key, value in scores_env.items():
    if value: print(key, np.average(value), np.median(value), len(value))
print('\n[1.4] Average Jaccard Scores by Time')
for key, value in scores_time.items():
    if value: print(key, np.average(value), np.median(value), len(value))

[1.1] Average Jaccard Scores by Class
borrachas 0.5325306585012034 0.547205548427838 16
cartas 0.9058389493609182 0.9706261541260099 16
celulares 0.8502312026181853 0.9368360033126877 16
conchas 0.7733642742648823 0.9667886576364418 16
copos 0.6024580314374761 0.5824047080315682 16
dados 0.5308388105615046 0.5563162989473909 16
estatuas 0.7072930525141282 0.8085422818989125 16
lapis 0.6340114586569175 0.647564594972067 16
tubos 0.7301262597442238 0.8325100673797264 16
vasos 0.7167864354104434 0.6853404243119419 16

[1.2] Average Jaccard Scores by Background
branco 0.7678914291822809 0.7893892238864034 63
preto 0.8631240744936604 0.9237565684956618 53
verde 0.4002938668743281 0.4271427754788698 44

[1.3] Average Jaccard Scores by Environment
exterior 0.7185712658490878 0.7669410956585798 79
interior 0.6786239028029654 0.7301627070683252 81

[1.4] Average Jaccard Scores by Time
dia 0.6708893759064378 0.6927218929180226 120
noite 0.7807235255086399 0.8947314397427242 40
